In [1]:
%cd ..

/home/den/dev/git/ozon-e-cup-2025


In [ ]:
import json
from pathlib import Path

import numpy as np
import pandas as pd

In [ ]:
embs_train = np.load("data/clip_image_embs_vit16.npy")
embs_test = np.load("data/clip_image_embs_vit16_test.npy")

idx2ItemID_train = {}
with open("data/clip_image_ids_vit16.jsonl", "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        obj = json.loads(line)
        idx2ItemID_train[idx] = int(Path(obj["path"].split("\\")[-1]).stem)

idx2ItemID_test = {}
with open("data/clip_image_ids_vit16_test.jsonl", "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        obj = json.loads(line)
        idx2ItemID_test[idx] = int(Path(obj["path"].split("\\")[-1]).stem)

In [ ]:
# Get ItemID to id mapping
train_meta = pd.read_csv("data/1__train_with_flags.csv", usecols=["id", "ItemID"])
test_meta = pd.read_csv("data/1__test_with_flags.csv", usecols=["id", "ItemID"])

ItemID2id_train = train_meta.set_index("ItemID")["id"].to_dict()
ItemID2id_test = test_meta.set_index("ItemID")["id"].to_dict()

In [ ]:
assert len(idx2ItemID_train.values()) == len(set(idx2ItemID_train.values()))
assert len(idx2ItemID_test.values()) == len(set(idx2ItemID_test.values()))

assert set(idx2ItemID_train.values()) <= ItemID2id_train.keys()
assert set(idx2ItemID_test.values()) <= ItemID2id_test.keys()

In [ ]:
print(f"Rows with images in train: {len(idx2ItemID_train) / len(train_meta) * 100:.2f}%")
print(f"Rows with images in test: {len(idx2ItemID_test) / len(test_meta) * 100:.2f}%")

Rows with images in train: 99.63%
Rows with images in test: 99.89%


In [ ]:
# Create id2idx mapping
id2idx_train = {ItemID2id_train[item_id]: idx for idx, item_id in idx2ItemID_train.items()}
id2idx_test = {ItemID2id_test[item_id]: idx for idx, item_id in idx2ItemID_test.items()}

In [ ]:
train_ids = np.load("data/train_ids.npy")
val_ids = np.load("data/val_ids.npy")
test_ids = np.load("data/test_ids.npy")

In [ ]:
real_img_embs_train = []
real_id2idx_img_embs_train = {}

for i, id in enumerate(train_ids):
    if id in id2idx_train:
        emb = embs_train[id2idx_train[id]]
    else:
        emb = np.zeros(512)
    real_img_embs_train.append(emb)
    real_id2idx_img_embs_train[int(id)] = i

In [ ]:
real_img_embs_val = []
real_id2idx_img_embs_val = {}

for i, id in enumerate(val_ids):
    if id in id2idx_train:
        emb = embs_train[id2idx_train[id]]
    else:
        emb = np.zeros(512)
    real_img_embs_val.append(emb)
    real_id2idx_img_embs_val[int(id)] = i

In [ ]:
real_img_embs_test = []
real_id2idx_img_embs_test = {}

for i, id in enumerate(test_ids):
    if id in id2idx_test:
        emb = embs_test[id2idx_test[id]]
    else:
        emb = np.zeros(512)
    real_img_embs_test.append(emb)
    real_id2idx_img_embs_test[int(id)] = i

In [ ]:
real_img_embs_train = np.array(real_img_embs_train)
real_img_embs_val = np.array(real_img_embs_val)
real_img_embs_test = np.array(real_img_embs_test)

In [ ]:
np.save("data/img_embs_train.npy", real_img_embs_train)
np.save("data/img_embs_val.npy", real_img_embs_val)
np.save("data/img_embs_test.npy", real_img_embs_test)

In [21]:
json.dump(real_id2idx_img_embs_train, open("data/id2idx_img_embs_train.json", "w"))
json.dump(real_id2idx_img_embs_val, open("data/id2idx_img_embs_val.json", "w"))
json.dump(real_id2idx_img_embs_test, open("data/id2idx_img_embs_test.json", "w"))